In [43]:
# We'll use the previously defined classes to construct the game tree as shown in the user's uploaded image.

class TreeNode:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

class BinaryTree:
    def __init__(self, root_value=None):
        self.root = TreeNode(root_value) if root_value is not None else None

    def insert_left(self, current_node, value):
        if current_node.left is None:
            current_node.left = TreeNode(value)
        else:
            new_node = TreeNode(value)
            new_node.left = current_node.left
            current_node.left = new_node

    def insert_right(self, current_node, value):
        if current_node.right is None:
            current_node.right = TreeNode(value)
        else:
            new_node = TreeNode(value)
            new_node.right = current_node.right
            current_node.right = new_node

In [46]:
# Create the root of the tree
tree = BinaryTree()

# Level 1 (Root) MAX
tree.root = TreeNode(None)

# Level 2 MIN
tree.root.left = TreeNode(5)
tree.root.right = TreeNode(None)

# Level 3 MAX

tree.root.right.left = TreeNode(None)
tree.root.right.right = TreeNode(None)

# Level 4 MAX
tree.root.right.left.left = TreeNode(1)
tree.root.right.left.right = TreeNode(None) 
tree.root.right.right.left = TreeNode(5)
tree.root.right.right.right = TreeNode(None)

# Level 5 MIN
tree.root.right.left.right.left = TreeNode(4)
tree.root.right.left.right.right = TreeNode(2)
tree.root.right.right.right.left = TreeNode(4)
tree.root.right.right.right.right = TreeNode(3)


def print_tree_like_structure(node, indent="", last='updown'):

    if node != None:
        print(indent, end='')

        if last == 'updown': 
            print('Root----', end='')
            indent += "     "
        elif last == 'right': 
            print('R----', end='')
            indent += "|    "
        elif last == 'left': 
            print('L----', end='')
            indent += "     "

        print(node.value)

        print_tree_like_structure(node.left, indent, 'left')
        print_tree_like_structure(node.right, indent, 'right')

# Now let's print the tree to see its structure
print_tree_like_structure(tree.root)


Root----None
     L----5
     R----None
     |    L----None
     |         L----1
     |         R----None
     |         |    L----4
     |         |    R----2
     |    R----None
     |    |    L----5
     |    |    R----None
     |    |    |    L----4
     |    |    |    R----3


In [51]:
# Minimax function
def minimax(node, depth, is_maximizing_player):
    # If the node is a leaf node, return its value
    if node.left is None and node.right is None:
        return node.value

    # If the current level is a maximizer
    if is_maximizing_player:
        best_val = float('-inf') # Negative infinity

        # Recur for left and right children and choose the maximum value
        if node.left:
            best_val = max(best_val, minimax(node.left, depth + 1, False))
        if node.right:
            best_val = max(best_val, minimax(node.right, depth + 1, False))

        return best_val

    else: # If the current level is a minimizer
        best_val = float('inf') # Positive infinity

        # Recur for left and right children and choose the minimum value
        if node.left:
            best_val = min(best_val, minimax(node.left, depth + 1, True))
        if node.right:
            best_val = min(best_val, minimax(node.right, depth + 1, True))

        return best_val

# Call the minimax function
# Assuming the root is at depth 0 and is a maximizer
minimax_value = minimax(tree.root, 0, True)
print("The Minimax value of the root is:", minimax_value)


The Minimax value of the root is: 5


In [54]:
def alpha_beta_pruning(node, depth, alpha, beta, is_maximizing_player, pruned_nodes):
    # If node is a leaf node, return its value
    if node.left is None and node.right is None:
        return node.value

    if is_maximizing_player:
        max_eval = float('-inf')
        if node.left:
            eval = alpha_beta_pruning(node.left, depth + 1, alpha, beta, False, pruned_nodes)
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                if node.right:
                    pruned_nodes.append([f'depth: {depth}', node.right.value])  # Prune right child
                return max_eval  # Return early as further exploration is unnecessary

        if node.right:
            eval = alpha_beta_pruning(node.right, depth + 1, alpha, beta, False, pruned_nodes)
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha and node.left:
                pruned_nodes.append([f'depth: {depth}', node.right.left])  # Prune left child
        return max_eval
    else:
        min_eval = float('inf')
        if node.left:
            eval = alpha_beta_pruning(node.left, depth + 1, alpha, beta, True, pruned_nodes)
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                if node.right:
                    pruned_nodes.append([f'depth: {depth}', node.right.value])  # Prune right child
                return min_eval  # Return early as further exploration is unnecessary

        if node.right:
            eval = alpha_beta_pruning(node.right, depth + 1, alpha, beta, True, pruned_nodes)
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha and node.left:
                pruned_nodes.append([f'depth: {depth}', node.left.value])  # Prune left child
        return min_eval

# Example usage
pruned_nodes = []
result = alpha_beta_pruning(tree.root, 0, float('-inf'), float('inf'), True, pruned_nodes)
print("Result:", result)
print("Pruned nodes:", pruned_nodes)


Result: 5
Pruned nodes: [['depth: 3', 2], ['depth: 1', 5]]


In [53]:
# We will update the minimax function to set the value of the None nodes to the min-max value determined by the algorithm.

def minimax(node, depth):
    # Terminal node (leaf node), return its value
    if node.left is None and node.right is None:
        return node.value

    if depth % 2 == 0:  # Even depth, MAX level
        best_value = float('-inf')

        # Recur for left and right children and choose the maximum value
        if node.left is not None:
            best_value = max(best_value, minimax(node.left, depth + 1))
        if node.right is not None:
            best_value = max(best_value, minimax(node.right, depth + 1))

        node.value = best_value  # Update the node's value to the best achievable for MAX
        return best_value
    else:  # Odd depth, MIN level
        best_value = float('inf')

        # Recur for left and right children and choose the minimum value
        if node.left is not None:
            best_value = min(best_value, minimax(node.left, depth + 1))
        if node.right is not None:
            best_value = min(best_value, minimax(node.right, depth + 1))

        node.value = best_value  # Update the node's value to the best achievable for MIN
        return best_value

# Apply the minimax algorithm to the tree
minimax(tree.root, 0)  # Start from the root (level 0), which is a MAX node

# After applying the minimax algorithm, the None nodes will have the min-max values.
# Let's define the function to print the tree with the updated values.

def print_tree_with_values(node, indent="", last='updown'):
    if node != None:
        print(indent, end='')

        if last == 'updown': 
            print('Root----', end='')
            indent += "     "
        elif last == 'right': 
            print('R----', end='')
            indent += "|    "
        elif last == 'left': 
            print('L----', end='')
            indent += "     "

        print(node.value)

        print_tree_with_values(node.left, indent, 'left')
        print_tree_with_values(node.right, indent, 'right')

# Print the tree to check the updated values
print_tree_with_values(tree.root)


Root----5
     L----5
     R----2
     |    L----2
     |         L----1
     |         R----2
     |         |    L----4
     |         |    R----2
     |    R----5
     |    |    L----5
     |    |    R----3
     |    |    |    L----4
     |    |    |    R----3


In [10]:


def alpha_beta_pruning(node, depth, alpha, beta, maximizing_player):
    if node is None:
        return float('-inf') if maximizing_player else float('inf')

    # If it is a terminal node (leaf node), return its value
    if node.left is None and node.right is None:
        return node.value

    if maximizing_player:
        best_value = float('-inf')
        # Recur for left and right children
        if node.left is not None:
            best_value = max(best_value, alpha_beta_pruning(node.left, depth + 1, alpha, beta, False))
            alpha = max(alpha, best_value)
            if beta <= alpha:
                return best_value  # Beta cut-off
        if node.right is not None:
            best_value = max(best_value, alpha_beta_pruning(node.right, depth + 1, alpha, beta, False))
            alpha = max(alpha, best_value)
            if beta <= alpha:
                return best_value  # Beta cut-off
        return best_value
    else:
        best_value = float('inf')
        # Recur for left and right children
        if node.left is not None:
            best_value = min(best_value, alpha_beta_pruning(node.left, depth + 1, alpha, beta, True))
            beta = min(beta, best_value)
            if beta <= alpha:
                return best_value  # Alpha cut-off
        if node.right is not None:
            best_value = min(best_value, alpha_beta_pruning(node.right, depth + 1, alpha, beta, True))
            beta = min(beta, best_value)
            if beta <= alpha:
                return best_value  # Alpha cut-off
        return best_value

# Apply alpha-beta pruning to the tree
alpha_beta_value = alpha_beta_pruning(tree.root, 0, float('-inf'), float('inf'), True)

# Print the value after alpha-beta pruning
alpha_beta_value


5

In [11]:
# To find the nodes that are not explored by the alpha-beta pruning method, we will modify the alpha-beta
# function to collect the pruned nodes.

pruned_nodes = []  # List to hold the values of the pruned nodes

def alpha_beta_pruning_with_trace(node, depth, alpha, beta, maximizing_player):
    if node is None:
        return float('-inf') if maximizing_player else float('inf')

    # If it is a terminal node (leaf node), return its value
    if node.left is None and node.right is None:
        return node.value

    global pruned_nodes

    if maximizing_player:
        best_value = float('-inf')
        # Recur for left and right children
        if node.left is not None:
            value = alpha_beta_pruning_with_trace(node.left, depth + 1, alpha, beta, False)
            best_value = max(best_value, value)
            alpha = max(alpha, best_value)
            if beta <= alpha and node.right is not None:
                pruned_nodes.append(node.right)  # Right subtree is pruned
                return best_value  # Beta cut-off
        if node.right is not None:
            value = alpha_beta_pruning_with_trace(node.right, depth + 1, alpha, beta, False)
            best_value = max(best_value, value)
            alpha = max(alpha, best_value)
            if beta <= alpha and node.left is not None:
                pruned_nodes.append(node.left)  # Left subtree is pruned
                return best_value  # Beta cut-off
        return best_value
    else:
        best_value = float('inf')
        # Recur for left and right children
        if node.left is not None:
            value = alpha_beta_pruning_with_trace(node.left, depth + 1, alpha, beta, True)
            best_value = min(best_value, value)
            beta = min(beta, best_value)
            if beta <= alpha and node.right is not None:
                pruned_nodes.append(node.right)  # Right subtree is pruned
                return best_value  # Alpha cut-off
        if node.right is not None:
            value = alpha_beta_pruning_with_trace(node.right, depth + 1, alpha, beta, True)
            best_value = min(best_value, value)
            beta = min(beta, best_value)
            if beta <= alpha and node.left is not None:
                pruned_nodes.append(node.left)  # Left subtree is pruned
                return best_value  # Alpha cut-off
        return best_value

# Reset the pruned nodes list and run the modified alpha-beta pruning
pruned_nodes = []
alpha_beta_pruning_with_trace(tree.root, 0, float('-inf'), float('inf'), True)

# Now, let's collect the values of the pruned nodes
pruned_values = [node.value for node in pruned_nodes]

pruned_values


[2, 5]

In [45]:
# We need to track the alpha and beta values at each node.
# This function will perform the alpha-beta pruning and store the alpha and beta values at each node.

node_values = []

def alpha_beta_with_tracking(node, depth, alpha, beta, maximizing_player):
    if node is None:
        return None

    if node.left is None and node.right is None:
        node_values.append((node.value, alpha, beta))
        return node.value

    if maximizing_player:
        best_value = float('-inf')
        if node.left is not None:
            best_value = max(best_value, alpha_beta_with_tracking(node.left, depth + 1, alpha, beta, False))
            alpha = max(alpha, best_value)
        if node.right is not None:
            best_value = max(best_value, alpha_beta_with_tracking(node.right, depth + 1, alpha, beta, False))
            alpha = max(alpha, best_value)
        node_values.append((node.value, alpha, beta))
        return best_value
    else:
        best_value = float('inf')
        if node.left is not None:
            best_value = min(best_value, alpha_beta_with_tracking(node.left, depth + 1, alpha, beta, True))
            beta = min(beta, best_value)
        if node.right is not None:
            best_value = min(best_value, alpha_beta_with_tracking(node.right, depth + 1, alpha, beta, True))
            beta = min(beta, best_value)
        node_values.append((node.value, alpha, beta))
        return best_value

# Run the alpha-beta pruning with tracking on the tree
node_values = []
alpha_beta_with_tracking(tree.root, 0, float('-inf'), float('inf'), True)

# Filter out the pruned nodes and display the alpha-beta values of the remaining nodes
remaining_node_values = [val for val in node_values if val[0] not in pruned_values]

remaining_node_values


[(1, 5, inf),
 (4, 5, inf),
 (None, 5, 2),
 (None, 5, inf),
 (4, 5, 2),
 (3, 5, 2),
 (None, 5, 2),
 (None, 5, 2),
 (None, 5, 2),
 (None, 5, inf)]